In [28]:
import tensorflow as tf
import numpy as np
import pickle

In [30]:
train_data = pickle.load(open('../word2vec/anouymous_train_data_200.npz', 'rb'))
train_inputs = train_data['inputs']
train_targets = train_data['targets']

In [234]:
def param_init(name, shape):
    return tf.get_variable(name = name, initializer = tf.truncated_normal(shape), dtype = tf.float32)
    
def BasicLSTMCell(n_units):
        
    with tf.variable_scope('basic_lstm_cell'):
        wf = param_init('wf', [n_units, n_units])
        bf = param_init('bf', [n_units])
        wi = param_init('wi', [n_units, n_units])
        bi = param_init('bi', [n_units])
        wc = param_init('wc', [n_units, n_units])
        bc = param_init('bc', [n_units])
        wo = param_init('wo', [n_units, n_units])
        bo = param_init('bo', [n_units])
        W = tf.concat([wf, wi, wc, wo], 0)
        B = tf.concat([bf, bi, bc, bo], 0)   
#         c = tf.Variable(tf.zeros([n_units]), dtype = tf.float32)
#         h = tf.reshape(inputs[:,0,:] - inputs[:,0,:], [-1, 1, n_units])
    return W, B
    
def dynamic_lstm_layer(inputs, n_units, cell, batch_size, step_size):
    
    shape = inputs.shape.as_list()
#     batch_size = shape[0]
#     step_size = shape[1]
    n_units = shape[2]
    print('dynamic')
#     ht = tf.reshape(inputs[:,0,:] - inputs[:,0,:], [-1, 1, n_units])
    print('reshape')
    h = tf.expand_dims(inputs[:,0,:], 1)
    c = tf.Variable(tf.zeros([n_units]), dtype = tf.float32)
    
    def train_step(i):
#         print('--------------')
#         print(xt.shape)
#         print(ct.shape)
#         print(ht.shape)
        nonlocal h
#         print(h.shape)
        xt = inputs[:, i, :]
#         ct = 
        ht = tf.reshape(h[: -1, :], [-1, n_units])
        ct = c
        
        V = tf.transpose(tf.matmul(cell[0], tf.transpose(xt, [1, 0])) + tf.matmul(cell[0], tf.transpose(ht, [1,0])), [1,0]) + cell[1]
        ft, it, cp, ot = tf.split(V, num_or_size_splits=4, axis=1)
        ft = tf.sigmoid(ft)
        it = tf.sigmoid(it)
        cp = tf.tanh(c)
        ot = tf.sigmoid(ot)
        new_c = tf.reshape(tf.reduce_mean(ft * tf.tile(tf.expand_dims(ct, 1), [batch_size, 1]) + it * cp, 0), [1, n_units])
        new_h = ot * tf.tile(tf.tanh(tf.expand_dims(ct, 1)), [batch_size, 1])
        h = tf.concat([h, tf.expand_dims(ht, 1)], 1)
        print('finished')
#         print(h.shape)
        return new_c, new_h
        
    def iteration(cell, step_size):
        print('iteration')
        i = tf.constant(0)
        ct, ht = tf.while_loop(lambda i: tf.less(i, step_size), lambda i: train_step(i), [i])
        
        return ct, ht
    
    new_c, new_ht = iteration(cell, step_size)
    return new_c, new_ht
    


In [235]:
graph = tf.Graph()
with graph.as_default():
    
    n_units = 200
    n_classes = 134
    n_hidden = 128
    
    inputs_placeholder = tf.placeholder(tf.float32, [None, None, n_units])
    targets_placeholder = tf.placeholder(tf.float32, [None, n_classes])
    step_size = tf.placeholder(tf.int32)
    batch_size = tf.placeholder(tf.int32)
    
    states, outputs = dynamic_lstm_layer(inputs=inputs_placeholder, n_units=n_units, 
                                         cell=BasicLSTMCell(n_units), step_size=step_size, batch_size = batch_size)
#     print(outputs)
    
    w1 = tf.Variable(tf.truncated_normal([n_units, n_units]), tf.float32)
    b1 = tf.Variable(tf.zeros([n_units]))
    w2 = tf.Variable(tf.truncated_normal([n_units, n_classes]), tf.float32)
    b2 = tf.Variable(tf.zeros([n_classes]))
#     print(outputs)
    l1 = tf.nn.relu(tf.add(tf.matmul(outputs[:, -1, :], w1), b1))
    o1 = tf.nn.relu(tf.add(tf.matmul(l1, w2), b2))
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=o1, labels=targets_placeholder))
    loss_sum = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=o1, labels=targets_placeholder))
    accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(o1, 1), tf.argmax(targets_placeholder, 1)), tf.float32))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    train_keys = train_inputs.keys()
    for i in range(100):
        accs = 0.0
        errs = 0.0
        n = 0
        for key in train_keys:
            batch_inputs = train_inputs[key]
            batch_targets = train_targets[key]
            size = len(batch_inputs)
#             print(key)
            if size == 0:
                continue
            feed_dict = {inputs_placeholder:batch_inputs, targets_placeholder:batch_targets, 
                         step_size:int(key.split('_')[0]), batch_size:size}
            _, acc, err = sess.run([optimizer, accuracy, loss_sum], feed_dict = feed_dict)
            accs += acc
            errs += err
            n += size
        print('|Training Epoch%d | ACC: %f | ERR: %f | SIZE: %d |'%(i, accs/n, errs/n, n))
    

dynamic
reshape
iteration
finished


ValueError: The two structures don't have the same number of elements. First structure: [<tf.Tensor 'while/Identity:0' shape=() dtype=int32>], second structure: [<tf.Tensor 'while/Reshape_1:0' shape=(1, 200) dtype=float32>, <tf.Tensor 'while/mul_2:0' shape=(?, 200) dtype=float32>].

In [132]:
i = tf.constant(0)
c = lambda i: tf.less(i, 10)
b = lambda i: tf.add(i, 1)
r = tf.while_loop(c, b, [i])
print(r)

Tensor("while_3/Exit:0", shape=(), dtype=int32)


In [185]:
def a():
    b = 1
    def c():
        print(b)


AttributeError: 'NoneType' object has no attribute 'c'

In [217]:
print(train_inputs['13_0'].shape)

(200, 13, 200)
